# Python `tinydb` Package

## Overview and Introduction
The Python `tinydb` package implements a document oriented database on a JSON file. Cannot be used a massive scale, but likely good enough to make this work initially and may be big enough for pretty much everything we need as most ML seems to consume flat files when doing linguistic stuff.

In [1]:
from tinydb import TinyDB, Query, where

First example from [HERE](https://tinydb.readthedocs.io/en/latest/getting-started.html#basic-usage). The `db.truncate()` function call clears the database completely. I needed to add this so that I could re-run this file a few times. Without it entries get written more than once to the file. On adding, every entry gets a unique identifier so it is possible to have multiple copies of the same entry.

### Create a Database (and Clear It)

In [2]:
db = TinyDB('db.json')
db.truncate()              # Clear the DB

Insert a few items as a test:

In [3]:
db.insert({'type': 'apple', 'count': 7})
db.insert({'type': 'peach', 'count': 3})
db.insert({'type': 'pear', 'count': 3})
db.insert({'type': 'banana', 'count': 12})

4

In [4]:
db.all()

[{'type': 'apple', 'count': 7},
 {'type': 'peach', 'count': 3},
 {'type': 'pear', 'count': 3},
 {'type': 'banana', 'count': 12}]

Databases are iterables (`iter(db)`):

In [5]:
for item in db:
    print(item)

{'type': 'apple', 'count': 7}
{'type': 'peach', 'count': 3}
{'type': 'pear', 'count': 3}
{'type': 'banana', 'count': 12}


In [6]:
for item in db:
    print(item['type'])

apple
peach
pear
banana


### Query Objects
Make a new `Query` object. Not sure how this works yet. Compare with `where()` function in the section below.

In [7]:
Fruit = Query()

In [8]:
db.search(Fruit.type == 'peach')

[{'type': 'peach', 'count': 3}]

In [9]:
db.search(Fruit.count == 5)      # Matches nothing

[]

In [10]:
db.search(Fruit.count == 3)

[{'type': 'peach', 'count': 3}, {'type': 'pear', 'count': 3}]

### Update Entries
Update some items, make other changes. Uses `Query()` objects.

In [11]:
db.all()

[{'type': 'apple', 'count': 7},
 {'type': 'peach', 'count': 3},
 {'type': 'pear', 'count': 3},
 {'type': 'banana', 'count': 12}]

In [12]:
db.update({'count': 10}, Fruit.type == 'apple')   # Change apple count to 10

[1]

In [13]:
db.all()

[{'type': 'apple', 'count': 10},
 {'type': 'peach', 'count': 3},
 {'type': 'pear', 'count': 3},
 {'type': 'banana', 'count': 12}]

Using the `query` object, we will get rid of all items with fewer than 5 items (so the peaches!):

In [14]:
db.remove(Fruit.count < 5)

[2, 3]

In [15]:
for item in db:
    print(item)

{'type': 'apple', 'count': 10}
{'type': 'banana', 'count': 12}


### Clear the Entire Database
To clear the whole database use `truncate` (why truncate?):

In [16]:
db.truncate()
db.all()

[]

## Review of Basic Commands
Here is a summary of what we did so far (copied from [HERE](https://tinydb.readthedocs.io/en/latest/getting-started.html#basic-usage)):

| Syntax | Description |
| ------: | :----------- |
| `db.insert(...)` | Insert a document |
| `db.all()` | Return all documents in the database as a list |
| `iter(db)` | Iterate over all documents in the database |
| `x = Query()` | Create a new `Query()` object |
| `Query().field == y` | Match documents with `field` equals `y` (usual comparisons work: `!=`, `>`, `>=`, `<`, `<=`) |
| `db.search(query)` | Use the `Query()` object to do a search for matches (`Query.field == Target`) |
| `db.update(fields, query)` | Update all corresponding `fields` in documents matching the `query` |
| `db.remove(query)` | Remove all documents matching the `query` |
| `db.truncate()` | Remove all documents in the database |

## "Advanced" Usage
Moving on to more advanced usage. See [HERE](https://tinydb.readthedocs.io/en/latest/usage.html) for details.

### Advanced Searching (Two Forms)
We will explore basic searching using a simple test database, created here:

In [17]:
ndb = TinyDB('users.json')
ndb.truncate()               # Added this to clean the DB on re-running the code!

In [18]:
ndb.insert({'name':'David Jones', 'birthday':{'month':'january', 'day': 12, 'year': 1990}})
ndb.insert({'name':'David Melbourne', 'birthday':{'month':'february', 'day': 19, 'year': 1971}})
ndb.insert({'name':'Mufti Jones', 'birthday':{'month':'december', 'day': 30, 'year': 1981}})
ndb.insert({'name':'dane double', 'birthday':{'month':'june', 'day': 22, 'year': 1977}})

4

In [19]:
ndb.all()

[{'name': 'David Jones',
  'birthday': {'month': 'january', 'day': 12, 'year': 1990}},
 {'name': 'David Melbourne',
  'birthday': {'month': 'february', 'day': 19, 'year': 1971}},
 {'name': 'Mufti Jones',
  'birthday': {'month': 'december', 'day': 30, 'year': 1981}},
 {'name': 'dane double',
  'birthday': {'month': 'june', 'day': 22, 'year': 1977}}]

#### Searching Using Query Objects
Start with advanced queries. Note that all DB fields should be valid Python identifiers so that the `Query.field` notation can be used; if not, `Query['field'] = value` is available as an alternative!

Search using a query:

In [20]:
User = Query()

In [21]:
ndb.search(User.name == 'David Jones')

[{'name': 'David Jones',
  'birthday': {'month': 'january', 'day': 12, 'year': 1990}}]

In [22]:
ndb.search(User.birthday.year == 1990)

[{'name': 'David Jones',
  'birthday': {'month': 'january', 'day': 12, 'year': 1990}}]

In [23]:
ndb.search(User.birthday.month == 'december')

[{'name': 'Mufti Jones',
  'birthday': {'month': 'december', 'day': 30, 'year': 1981}}]

#### Searching Using the `where()` Function
Search using the `where()` function (imported at top). This is a more traditional construct for searching. Less ORM.

In [24]:
ndb.search(where('name') == 'David Jones')

[{'name': 'David Jones',
  'birthday': {'month': 'january', 'day': 12, 'year': 1990}}]

In [25]:
ndb.search(where('birthday')['month'] == 'february')

[{'name': 'David Melbourne',
  'birthday': {'month': 'february', 'day': 19, 'year': 1971}}]

Compare this to the `Query['field'] = value` format mentioned above. Note that `where()` is essentially syntactic sugar for:

```Python
db.search(Query()['field'] == 'value')
```
#### Advanced Queries/Searches
These will show ways to build more complex search structures and use the `Query()` function format.

In [26]:
ndb.search(User.name.exists())

[{'name': 'David Jones',
  'birthday': {'month': 'january', 'day': 12, 'year': 1990}},
 {'name': 'David Melbourne',
  'birthday': {'month': 'february', 'day': 19, 'year': 1971}},
 {'name': 'Mufti Jones',
  'birthday': {'month': 'december', 'day': 30, 'year': 1981}},
 {'name': 'dane double',
  'birthday': {'month': 'june', 'day': 22, 'year': 1977}}]

The example below uses something from the `re` package. It is set in the `flags` options:

In [27]:
import re
ndb.search(User.name.matches('david JONES', flags=re.IGNORECASE))

[{'name': 'David Jones',
  'birthday': {'month': 'january', 'day': 12, 'year': 1990}}]

The method `.matches()` requires a whole string match. 

There are some other `re` tricks, using the `.search()` method that looks for **substring** matches in the fields:

In [28]:
ndb.search(User.name.search('david', flags=re.IGNORECASE))

[{'name': 'David Jones',
  'birthday': {'month': 'january', 'day': 12, 'year': 1990}},
 {'name': 'David Melbourne',
  'birthday': {'month': 'february', 'day': 19, 'year': 1971}}]

The `flags=re.IGNORECASE` option seems to work with **strings** as the search term, but not with regular expressions.

In [29]:
ndb.search(User.name.search('b+'))     # Search for names with at least one "b" in a substring

[{'name': 'David Melbourne',
  'birthday': {'month': 'february', 'day': 19, 'year': 1971}},
 {'name': 'dane double',
  'birthday': {'month': 'june', 'day': 22, 'year': 1977}}]

Unfortunately the `flags=re.IGNORECASE` seems to be broken in the `.search()` method:

In [30]:
ndb.search(User.name.search('D'))  # Note that this is **not** case insensitive

[{'name': 'David Jones',
  'birthday': {'month': 'january', 'day': 12, 'year': 1990}},
 {'name': 'David Melbourne',
  'birthday': {'month': 'february', 'day': 19, 'year': 1971}}]

In [31]:
ndb.search(User.name.search('D', flags=re.IGNORECASE))

[{'name': 'David Jones',
  'birthday': {'month': 'january', 'day': 12, 'year': 1990}},
 {'name': 'David Melbourne',
  'birthday': {'month': 'february', 'day': 19, 'year': 1971}}]

And yet:

In [32]:
ndb.search(User.name.search('d'))

[{'name': 'David Jones',
  'birthday': {'month': 'january', 'day': 12, 'year': 1990}},
 {'name': 'David Melbourne',
  'birthday': {'month': 'february', 'day': 19, 'year': 1971}},
 {'name': 'dane double',
  'birthday': {'month': 'june', 'day': 22, 'year': 1977}}]

In [33]:
ndb.search(User.name.search('[dD]'))

[{'name': 'David Jones',
  'birthday': {'month': 'january', 'day': 12, 'year': 1990}},
 {'name': 'David Melbourne',
  'birthday': {'month': 'february', 'day': 19, 'year': 1971}},
 {'name': 'dane double',
  'birthday': {'month': 'june', 'day': 22, 'year': 1977}}]

This seems to be a bug? Or there is a rule about lowercase being more generic than uppercase?

I really don't understand this yet.

Test functions:

In [34]:
test_func = lambda s: s == 'David Melbourne'

In [35]:
ndb.search(User.name.test(test_func))

[{'name': 'David Melbourne',
  'birthday': {'month': 'february', 'day': 19, 'year': 1971}}]

In [36]:
def new_test_func(s):
    return "david melbourne" == s.lower()   # Need return for this form of function definition

In [37]:
new_test_func("David Melbourne")

True

In [38]:
ndb.search(User.name.test(new_test_func))

[{'name': 'David Melbourne',
  'birthday': {'month': 'february', 'day': 19, 'year': 1971}}]

Searching using regular expressions. I am not very good with these, but let's see:

In [39]:
# re was imported above

ndb.insert({'name':'BBBBB', 'birthday':{'month':'june', 'day': 11, 'year': 1977}})

5

The following should be equivalent:

In [40]:
ndb.search(User.name.search('b+', flags=re.IGNORECASE))

[{'name': 'David Melbourne',
  'birthday': {'month': 'february', 'day': 19, 'year': 1971}},
 {'name': 'dane double',
  'birthday': {'month': 'june', 'day': 22, 'year': 1977}},
 {'name': 'BBBBB', 'birthday': {'month': 'june', 'day': 11, 'year': 1977}}]

In [41]:
ndb.search(User.name.search('B+', flags=re.IGNORECASE))

[{'name': 'David Melbourne',
  'birthday': {'month': 'february', 'day': 19, 'year': 1971}},
 {'name': 'dane double',
  'birthday': {'month': 'june', 'day': 22, 'year': 1977}},
 {'name': 'BBBBB', 'birthday': {'month': 'june', 'day': 11, 'year': 1977}}]

In [42]:
ndb.search(User.name.search('[bB]+'))

[{'name': 'David Melbourne',
  'birthday': {'month': 'february', 'day': 19, 'year': 1971}},
 {'name': 'dane double',
  'birthday': {'month': 'june', 'day': 22, 'year': 1977}},
 {'name': 'BBBBB', 'birthday': {'month': 'june', 'day': 11, 'year': 1977}}]

## Using Lists as Fields in the JSON
The main reason I want to use JSON and a document oriented database is to allow lists of ingredients for the spec of a cocktail.

In Python specs are held as `dict` objects and they get serialized to JSON easily. So here are some tests with lists.

With lists, you can use the `any()` and `all()` methods for search.

### Cocktail DB Spec (Test)

In [43]:
cocktailDB = TinyDB('cocktailDB.json')
cocktailDB.truncate()   

Here is the prototype JSON for the DB Specs:

+ name: Cocktail Name.
+ creator: Cocktail Creator (if known).
+ source: Original source of spec.
+ url: URL.
+ didactic: Recipe, didactic.
+ standardized: Recipe, standardized.
+ codexClass: Generic cocktail class. (Type of cocktail, generally, following Cocktail Codex: Old Fashioned, Martini, Daiquiri, Sidecar, Highball, Flip. Allow `None` or `Nil`.)
+ genericProcess: Generic process class. (Process class: built, stirred (built), stirred (mixing glass), shaken, blended, flash blended.)
+ genericComponent: Generic component list (serialized).
+ brandComponent: Preferred component list, when relevant, allow `None` or `Nil`.
+ milliters: Component amounts, in millitres. (Per IBA.)
+ glassware: Glassware.
+ localAuthority: Author (local authority for spec).

Measurements are a problem; we set target measurements to **ml**. However, converting measures from dashes to ml is a problem. But we do not want a free-for-all! 

In [44]:
cocktailDB.insert({'name': 'Old Fashioned',
                   'codexClass': 'Old Fashioned',
                   'genericProcess': 'built',
                   'genericComponent': ['bourbon', 'water', 'bitters', 'sugar'],
                   'measurementUnits': ['ml', 'ml', 'dash', 'ml'],
                   'measurementValues': [60, 5, 3, 5],
                   'brandComponent': ['Old Grand-Dad', None, 'Angostura', None],
                   'didacticRecipe': 'Muddle bitters and sugar in glass. Add bourbon and ice (preferably one large cube) and stir. Garnish.',
                   'garnish': 'Orange Peel',
                   'glassware': 'double old fashioned glass',
                   'ibaSpecExists': True,
                   'ibaSpecHere': False,
                   'creator': 'temps immémorial',
                   'wikiUrl': 'https://en.wikipedia.org/wiki/Old_fashioned_(cocktail)',
                   'url': 'https://www.liquor.com/recipes/bourbon-old-fashioned/',
                   'sourceAuthority': 'liquor.com',
                   'localAuthority': 'doctorambient'
                  })
cocktailDB.insert({'name': 'Suffering Bastard (1942)',
                   'codexClass': 'Sidecar',
                   'genericProcess': 'shaken',
                   'genericComponent': ['brandy', 'gin', 'lime juice', 'bitters', 'ginger beer'],
                   'measurementUnits': ['ml', 'ml', 'ml', 'dash', 'top'],
                   'measurementValues': [30, 30, 15, 2, 1],
                   'brandComponent': [None, None, None, 'Angostura', 'Fever Tree Ginger Beer'],
                   'didacticRecipe': 'Combine ingredients and shake; dump with gated finish.',
                   'garnish': 'Mint Sprig',
                   'glassware': 'Suffering Bastard Mug',
                   'ibaSpecExists': True,
                   'ibaSpecHere': True,
                   'creator': 'Joe Scialom',
                   'wikiUrl': 'https://en.wikipedia.org/wiki/Suffering_Bastard',
                   'url': 'https://www.liquor.com/recipes/suffering-bastard/',
                   'sourceAuthority': 'liquor.com',
                   'localAuthority': 'doctorambient'
                  })
cocktailDB.insert({'name': 'Gimlet (Fresh)',
                   'codexClass': 'Daiquiri',
                   'genericProcess': 'shaken',
                   'genericComponent': ['gin', 'lime juice', 'simple syrup'],
                   'measurementUnits': ['ml', 'ml', 'ml'],
                   'measurementValues': [60, 30, 22],
                   'brandComponent': ['Plymouth Gin', None, None],
                   'didacticRecipe': 'Combine ingredients and shake; double strain. Served up.',
                   'garnish': None,
                   'glassware': 'Nick and Nora',
                   'ibaSpecExists': False,
                   'creator': 'temps immémorial (1928)',
                   'wikiUrl': 'https://en.wikipedia.org/wiki/Gimlet_(cocktail)',
                   'url': None,
                   'altUrl': 'https://www.liquor.com/recipes/gimlet/',
                   'sourceAuthority': 'Cocktail Codex',
                   'sourceReference': 115,
                   'localAuthority': 'doctorambient'
                  })

3

In [45]:
cocktailDB.all()

[{'name': 'Old Fashioned',
  'codexClass': 'Old Fashioned',
  'genericProcess': 'built',
  'genericComponent': ['bourbon', 'water', 'bitters', 'sugar'],
  'measurementUnits': ['ml', 'ml', 'dash', 'ml'],
  'measurementValues': [60, 5, 3, 5],
  'brandComponent': ['Old Grand-Dad', None, 'Angostura', None],
  'didacticRecipe': 'Muddle bitters and sugar in glass. Add bourbon and ice (preferably one large cube) and stir. Garnish.',
  'garnish': 'Orange Peel',
  'glassware': 'double old fashioned glass',
  'ibaSpecExists': True,
  'ibaSpecHere': False,
  'creator': 'temps immémorial',
  'wikiUrl': 'https://en.wikipedia.org/wiki/Old_fashioned_(cocktail)',
  'url': 'https://www.liquor.com/recipes/bourbon-old-fashioned/',
  'sourceAuthority': 'liquor.com',
  'localAuthority': 'doctorambient'},
 {'name': 'Suffering Bastard (1942)',
  'codexClass': 'Sidecar',
  'genericProcess': 'shaken',
  'genericComponent': ['brandy',
   'gin',
   'lime juice',
   'bitters',
   'ginger beer'],
  'measurementUni

Aside some examples of iteration and accessing fields:

In [46]:
for spec in cocktailDB:
    print('{:>35}\t{}'.format(spec['name'], spec['garnish']))

                      Old Fashioned	Orange Peel
           Suffering Bastard (1942)	Mint Sprig
                     Gimlet (Fresh)	None


In [47]:
for spec in cocktailDB:
    print(type(spec))

<class 'tinydb.table.Document'>
<class 'tinydb.table.Document'>
<class 'tinydb.table.Document'>


In [48]:
Drink = Query()   # Create a new Query object as before

for spec in cocktailDB.search(Drink.genericProcess == 'shaken'):
    print('{:>25}:\t{}'.format(spec['name'], spec['genericComponent']))

 Suffering Bastard (1942):	['brandy', 'gin', 'lime juice', 'bitters', 'ginger beer']
           Gimlet (Fresh):	['gin', 'lime juice', 'simple syrup']


Some queries:

In [49]:
Drink = Query()

In [50]:
# Search for bourbon based drinks:

x = cocktailDB.search(Drink.genericComponent.any(['bourbon']))
print(len(x))

for spec in x:
    print(spec['name'])

1
Old Fashioned


In [51]:
def drinks_by_spirit(spirit):          # Implicit global for the DB
    l = cocktailDB.search(Drink.genericComponent.any([spirit]))
    return [spec['name'] for spec in l] 

In [52]:
drinks_by_spirit('gin')

['Suffering Bastard (1942)', 'Gimlet (Fresh)']

In [53]:
drinks_by_spirit('bourbon')

['Old Fashioned']

In [54]:
drinks_by_spirit('brandy')

['Suffering Bastard (1942)']